In [1]:
pip install fastapi uvicorn requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.9 MB/s eta 0:00:00


In [3]:
from fastapi import FastAPI, HTTPException
import requests
from typing import List, Dict
import uvicorn
import nest_asyncio

# Apply the nest_asyncio patch to allow running in a notebook
nest_asyncio.apply()

app = FastAPI()

# Configuration
WINDOW_SIZE = 10
TIMEOUT = 0.5  # 500 ms timeout for external requests
window = []  # Window to store numbers

# URLs for the third-party test server APIs
API_URLS = {
    'p': "http://20.244.56.144/test/primes",
    'f': "http://20.244.56.144/test/fibo",
    'e': "http://20.244.56.144/test/even",
    'r': "http://20.244.56.144/test/rand"
}

def fetch_numbers_from_api(url: str) -> List[int]:
    """
    Fetch numbers from the test server API with a timeout.
    """
    try:
        response = requests.get(url, timeout=TIMEOUT)
        response.raise_for_status()
        return response.json().get("numbers", [])
    except (requests.exceptions.Timeout, requests.exceptions.RequestException):
        return []

@app.get("/numbers/{numberid}")
async def get_numbers(numberid: str) -> Dict:
    """
    Fetch numbers based on numberid, maintain a sliding window, and calculate the average.
    """
    if numberid not in API_URLS:
        raise HTTPException(status_code=400, detail="Invalid number ID")

    # Record the previous state of the window
    window_prev_state = list(window)

    # Fetch numbers from the appropriate test server API
    numbers = fetch_numbers_from_api(API_URLS[numberid])

    # Filter out duplicate numbers
    unique_numbers = [num for num in numbers if num not in window]

    # Add unique numbers to the window, maintaining the window size
    for num in unique_numbers:
        if len(window) < WINDOW_SIZE:
            window.append(num)
        else:
            window.pop(0)  # Remove the oldest number to maintain window size
            window.append(num)

    # Calculate the average of the numbers in the window
    if window:
        avg = sum(window) / len(window)
    else:
        avg = 0.0

    # Prepare the response
    response = {
        "windowPrevState": window_prev_state,
        "windowCurrState": list(window),
        "numbers": numbers,
        "avg": round(avg, 2)
    }

    return response

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=9876)


INFO:     Started server process [3235]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:9876 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [3235]
